# Computation of the empirical probability content of a depth contour

Import packages

In [1]:
import mallows_kendall as mk
import numpy as np
import itertools as it
import scipy as sp
import random
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import math

### The C++ load and verification part

Compile C++ codes if necessary

In [2]:
# Compilation
#import os
#os.system('clang++ -mmacosx-version-min=10.13 -std=gnu++11 -fPIC  -Wall -g -O2  -c rankDepth.cpp -o rankDepth.o')
#os.system('clang++ -mmacosx-version-min=10.13 -std=gnu++11 -dynamiclib -Wl,-headerpad_max_install_names -undefined dynamic_lookup -single_module -multiply_defined suppress -o rankDepth.so rankDepth.o')
# Restart the kernel after this line

Import (and test) the dynamic library

In [3]:
# Import and Test 1
from ctypes import *
cdll.LoadLibrary('rankDepth.so')
libc = CDLL('rankDepth.so')
libc.testFunc()

43

In [4]:
# Test 2
val = np.array([5.1])
obj = np.array([0.1, 1.1, 2.1, 3.1, 4.1, 5.1, 6.1, 7.1, 8.1, 9.1])
n = 10
outp = np.array([1.1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
res2 = libc.testFuncCmp(c_void_p(val.ctypes.data),
                       c_void_p(obj.ctypes.data),
                       c_int(n),
                       c_void_p(outp.ctypes.data))
print(res2)
print(outp)

0
[  1.1   0.    0.    0.    0.    0.    0.    0.    0.  142. ]


### Experiment with changing N

In [5]:
# Define data settings and prepare structures
np.random.seed(1)
n = 10
print(math.factorial(n))
print(mk.max_dist(n))
models = ["Mallows", "MallowsMix", "..."]
ms = [10, 50, 250]
thetas = [0.25]
# Plot the graphic
with PdfPages("pic-regions-cardinal-n" + str(n) + "-" + models[1] + ".pdf") as pdf:
    # Loop for the variance
    for i in range(len(thetas)):
        # Loop for the numbers of observaitons
        for j in range(len(ms)):
            m = ms[j]
            plt.figure(1, figsize=(5, 5))
            plt.plot()
            plt.title("Cardinality of depth contours, theta = " + str(thetas[i]) + ", N = " + str(ms[j]))
            plt.xlabel("Empirical depth")
            plt.ylabel("Portion of the space in the depth region")
            plt.ylim(0, 1)
            plt.xlim(0.2, 0.8)
            #plt.xlim(0.525, 0.75)
            #plt.xticks([0.55, 0.6, 0.65, 0.7, 0.75])
            plt.grid()
            # Loop for the MC iterations
            for k in range(50):
                print("Starting: i = " + str(i) + ", j = " + str(j) + ", k = " + str(k))
                # Generate data
                # Model 1: Mallows-Kendal
                #cons1 = np.random.permutation(n)
                #empDist = mk.sampling_mm(m = m, n = n, theta = thetas[i], phi = None, s0 = cons1)
                # Model 2: Mallows mixture
                mTmp = int(m / 2)
                cons1 = np.random.permutation(n)
                empDist1 = mk.sampling_mm(m = mTmp, n = n, theta = thetas[i], phi = None, s0 = cons1)
                # Shift the center
                cons2 = np.array(cons1)
                tmpWhere1 = np.where(cons1 == 0)
                tmpWhere2 = np.where(cons1 == 3)
                cons2[tmpWhere1] = 3
                cons2[tmpWhere2] = 0
                print(mk.kendall_tau(cons2, cons1))
                empDist2 = mk.sampling_mm(m = mTmp, n = n, theta = thetas[i], phi = None, s0 = cons2)
                empDist = np.concatenate((np.array(empDist1, dtype = c_int), 
                                          np.array(empDist2, dtype = c_int)))
                # Model 3: 
                # Prepare the memory
                nMaxDepths = np.array((mk.max_dist(n) + 1) * empDist.shape[0], dtype = c_int)
                maxDepths = np.array(range(nMaxDepths), dtype = c_double)
                maxCards = np.array(range(nMaxDepths), dtype = c_int)
                # Calculate the depths
                res1 = libc.depthCardsKendallTau(c_void_p(empDist.ctypes.data),
                                                 c_int(n),
                                                 c_int(m),
                                                 c_void_p(nMaxDepths.ctypes.data),
                                                 c_void_p(maxDepths.ctypes.data),
                                                 c_void_p(maxCards.ctypes.data))
                # Postprocess the output
                depths = (mk.max_dist(n) - maxDepths / m) / mk.max_dist(n)
                depths = depths[maxCards > 0.5]
                cards = np.cumsum(maxCards[maxCards > 0.5]) / math.factorial(n)
                # Plot the line
                plt.plot(depths, cards, ls = 'solid', color = 'black', lw = 0.5)
                print("Finished.")
            pdf.savefig()
            plt.close()

3628800
45
Starting: i = 0, j = 0, k = 0
5
Finished.
Starting: i = 0, j = 0, k = 1
5
Finished.
Starting: i = 0, j = 0, k = 2
5
Finished.
Starting: i = 0, j = 0, k = 3
5
Finished.
Starting: i = 0, j = 0, k = 4
5
Finished.
Starting: i = 0, j = 0, k = 5
5
Finished.
Starting: i = 0, j = 0, k = 6
5
Finished.
Starting: i = 0, j = 0, k = 7
5
Finished.
Starting: i = 0, j = 0, k = 8
5
Finished.
Starting: i = 0, j = 0, k = 9
5
Finished.
Starting: i = 0, j = 0, k = 10
5
Finished.
Starting: i = 0, j = 0, k = 11
5
Finished.
Starting: i = 0, j = 0, k = 12
5
Finished.
Starting: i = 0, j = 0, k = 13
5
Finished.
Starting: i = 0, j = 0, k = 14
5
Finished.
Starting: i = 0, j = 0, k = 15
5
Finished.
Starting: i = 0, j = 0, k = 16
5
Finished.
Starting: i = 0, j = 0, k = 17
5
Finished.
Starting: i = 0, j = 0, k = 18
5
Finished.
Starting: i = 0, j = 0, k = 19
5
Finished.
Starting: i = 0, j = 0, k = 20
5
Finished.
Starting: i = 0, j = 0, k = 21
5
Finished.
Starting: i = 0, j = 0, k = 22
5
Finished.
Starting: 

# Some random tries: to delete a proiri

In [ ]:
n = 7
m = 50
thetas = [0.25]
i = 0
cons1 = np.random.permutation(n)
empDist1 = mk.sampling_mm(m = m, n = n, theta = thetas[i], phi = None, s0 = cons1)
nMaxDepths = np.array((mk.max_dist(n) + 1) * empDist1.shape[0], dtype = c_int)
maxDepths = np.array(range(nMaxDepths), dtype = c_double)
maxCards = np.array(range(nMaxDepths), dtype = c_int)
res1 = libc.depthCardsKendallTau(c_void_p(empDist1.ctypes.data),
                                 c_int(n),
                                 c_int(m),
                                 c_void_p(nMaxDepths.ctypes.data),
                                 c_void_p(maxDepths.ctypes.data),
                                 c_void_p(maxCards.ctypes.data))
depths = (mk.max_dist(n) - maxDepths / m) / mk.max_dist(n)
depths = depths[maxCards > 0.5]
cards = np.cumsum(maxCards[maxCards > 0.5]) / math.factorial(n)

In [ ]:
plt.figure(1, figsize=(5, 5))
plt.plot()
plt.title("Cardinality of depth contours, theta = " + str(thetas[i]) + ", N = " + str(ms[j]))
plt.xlabel("Empirical depth")
plt.ylabel("Portion of the space in the depth region")
plt.ylim(0, 1)
plt.xlim(0.525, 0.75)
plt.xticks([0.55, 0.6, 0.65, 0.7, 0.75])
plt.grid()

In [ ]:
import mallows_kendall as mk
n = 7
cons1 = np.random.permutation(n)
print(cons1)
cons2 = np.array(cons1)
rDist = 1
tmpWhere1 = np.where(cons1 == 0)
tmpWhere2 = np.where(cons1 == 3)
cons2[tmpWhere1] = 3
cons2[tmpWhere2] = 0
print(cons1)
print(cons2)
mk.kendall_tau(cons2, cons1)